# ETL Automation - IMDb Database

## 0.0. Imports

In [1]:
import pandas as pd

import os
import shutil
import sqlite3
import logging
import requests

C:\Users\Caio\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1.0. Logging Configuration

In [2]:
#Logging Configuration
log_format = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.DEBUG, format=log_format)

## 2.0. Data Extraction

In [3]:
#Definition for Data Scraping at the website
DATABASE = "imdb_data.db"
DIRECTORY = "data"
URL = "https://datasets.imdbws.com/"
FILES = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz"
]

In [4]:
#Function for Data Extraction
def extract(
    data_url: str = URL,
    files: list = FILES,
    data_dir: str = DIRECTORY,
) -> None:
    """
    TODO: Docstring.
    """

    os.makedirs(data_dir, exist_ok=True)

    for file in files:
        url = data_url + file
        dir_path = os.path.join(data_dir, file)

        if not os.path.exists(dir_path):
            logging.info(f"Downloading {file}...")
            response = requests.get(url, stream=True)

            if response.status_code == 200:
                with open(dir_path, "wb") as f:
                    shutil.copyfileobj(response.raw, f)

                logging.info(f"{file} successfully downloaded!")

            else:
                logging.error(f"Fail downloading {file}! Status Code: {response.status_code}")

            del response
        else:
            logging.info(f"{file} already exists! Skipping download...")

    logging.info("Download Finished!")

## 3.0. Data Transform

In [5]:
#Function for Data Transform
def transform(
    files: list = FILES,
) -> None:
    """
    TODO: Docstring.
    """

    data_dir = "data"
    df_dir = os.path.join(data_dir, "processed")

    os.makedirs(df_dir, exist_ok=True)

    for file in files:
        dir_path = os.path.join(data_dir, file)

        if os.path.isfile(dir_path) and file.endswith(".gz"):
            logging.debug(f"Reading and processing data {file}...")

            df = pd.read_csv(
                dir_path,
                sep="\t",
                compression="gzip",
                low_memory=False,
                nrows=1_000
                #chunksize=1_000_000
            )

            df.replace({"\\N": None}, inplace=True)

            dest_dir = os.path.join(df_dir, file[:-3])
            df.to_csv(dest_dir, sep="\t", index=False)

            logging.debug(f"Processing done for {file}. Processed file saved in {dest_dir}!")

            #Remove file after processing
            #os.remove(dir_path)

    logging.info("All files processed and saved on folder 'processed'!")

## 4.0. Data Load

In [6]:
#Function for Data Load
def load(
    conn
) -> None:
    """
    TODO: Docstring.
    """

    df_dir = os.path.join("data", "processed")

    files = os.listdir(df_dir)

    for file in files:
        file_path = os.path.join(df_dir, file)

        if os.path.isfile(file_path) and file.endswith(".tsv"):
            df = pd.read_csv(file_path, sep="\t", low_memory=False)

            table_name = os.path.splitext(file)[0]
            table_name = table_name.replace(".", "_").replace("-", "_")

            df.to_sql(table_name, conn, index=False, if_exists="replace")

            logging.info(f"File {file} saved as table {table_name} to the database.")

            #Remove file path after load
            #os.remove(file_path)

    logging.info("All files have been saved to the database!")

## 5.0. Data Testing

In [ ]:
#Function for Data Testing and Set Analytic Views
def analytic_views(
    conn
) -> None:
    """
    TODO: Docstring.
    """

    title_analytics = """
    CREATE TABLE IF NOT EXISTS title_analytics AS

    WITH 
    participants AS (
        SELECT
            tconst,
            COUNT(DISTINCT nconst) as qtParticipants
        
        FROM title_principals
        
        GROUP BY 1
    )

    SELECT
        tb.tconst,
        tb.titleType,
        tb.originalTitle,
        tb.startYear,
        tb.endYear,
        tb.genres,
        tr.averageRating,
        tr.numVotes,
        tp.qtParticipants

    FROM title_basics tb 

    LEFT JOIN title_ratings tr
        ON tr.tconst = tb.tconst

    LEFT JOIN participants tp
        ON tp.tconst = tb.tconst
    """

    participants_analytics = """
    CREATE TABLE IF NOT EXISTS participants_analytics AS

    SELECT
        tp.nconst,
        tp.tconst,
        tp.ordering,
        tp.category,
        tb.genres

    FROM title_principals tp

    LEFT JOIN title_basics tb
        ON tb.tconst = tp.tconst
    """

    queries = [
        "DROP TABLE IF EXISTS title_analytics",
        "DROP TABLE IF EXISTS participants_analytics",
        title_analytics,
        participants_analytics
    ]

    logging.info("Saving analytical tables to the database...")

    for query in queries:
        conn.execute(query)
        

    logging.info("Analytical tables successfully created!")

    print("End of ETL Process!")

## 6.0. ETL Execution

In [ ]:
if __name__ == "__main__":
    extract()
    transform()

    conn = sqlite3.connect(DATABASE)

    load(conn=conn)
    analytic_views(conn=conn)

    conn.close()

2025-06-09 21:44:26,870 - INFO - Downloading name.basics.tsv.gz...
2025-06-09 21:44:26,874 - DEBUG - Starting new HTTPS connection (1): datasets.imdbws.com:443
2025-06-09 21:44:27,405 - DEBUG - https://datasets.imdbws.com:443 "GET /name.basics.tsv.gz HTTP/1.1" 200 285721152
2025-06-09 21:44:46,086 - INFO - name.basics.tsv.gz successfully downloaded!
2025-06-09 21:44:46,087 - INFO - Downloading title.akas.tsv.gz...
2025-06-09 21:44:46,089 - DEBUG - Starting new HTTPS connection (1): datasets.imdbws.com:443
2025-06-09 21:44:46,146 - DEBUG - https://datasets.imdbws.com:443 "GET /title.akas.tsv.gz HTTP/1.1" 200 454634118
2025-06-09 21:45:15,896 - INFO - title.akas.tsv.gz successfully downloaded!
2025-06-09 21:45:15,896 - INFO - Downloading title.basics.tsv.gz...
2025-06-09 21:45:15,898 - DEBUG - Starting new HTTPS connection (1): datasets.imdbws.com:443
2025-06-09 21:45:15,946 - DEBUG - https://datasets.imdbws.com:443 "GET /title.basics.tsv.gz HTTP/1.1" 200 206922592
2025-06-09 21:45:29,50

End of ETL Process!
